<a href="https://colab.research.google.com/github/JayashreeNagaraj/Capstone/blob/main/ITTicket_Fasttext.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


# New section

Plain Model (without lemmatization/stop words removale and with): 0.616 , 0.62

Model with epoch 25 : 0.774, 0.807

Model with LR 1: 0.829, 0.83

Model with epoch and lr : 0.95, 0.94

Model with Ngrams : 0.905, 0.904

It appears as if model with epoch and lr have higher precision. But the prediction for data with lesser data seems not accurate in prediction. 




In [4]:
import os
os.chdir("/content/drive/My Drive/Colab Notebooks/CPSTN_NLP_GRP6A/")

In [5]:
!pip install fasttext

     |████████████████████████████████| 71kB 3.2MB/s 
  Created wheel for fasttext: filename=fasttext-0.9.2-cp36-cp36m-linux_x86_64.whl size=3038010 sha256=5af72ab4b3f580929aabd23bb336fdf1c2db5ffa904b483f33f66b3df9ba0715
  Stored in directory: /root/.cache/pip/wheels/98/ba/7f/b154944a1cf5a8cee91c154b75231136cc3a3321ab0e30f592
Successfully built fasttext


In [6]:
import fasttext
import os
import numpy as np
import pandas as pd
import sys
from sklearn.model_selection import train_test_split
import copy

In [7]:
!ls
df = pd.read_excel("input_data.xlsx")
#df.apply(lambda x: x["Assignment group"] + " " + x["Description"])

 Bert				     ft_train.txt
 bert_old.py			     ft_train_v1.txt
 Capstone_yash.ipynb		     ft_train_v2.txt
'Copy of Webapp.ipynb'		     ft_train_v.txt
 data_modeled.csv		     ft_val1.txt
 data_modeled.gsheet		     ft_val2.txt
 Documents			     ft_val.txt
 fasttext_auto_tune_90accuracy.bin   inp_csv.csv
 fasttext.bin			     input_data.xlsx
 fasttext_lrpt1_e50.bin		     lstm_model
 fasttext_main.bin		     ML_Classifiers.ipynb
 ft_test1.txt			     multilingual_L-12_H-768_A-12.zip
 ft_test2.txt			     Notebooks
 ft_test.txt			     project_local.ipynb
 ft_train_1.txt			     xtest_conv.csv
 ft_train_2.txt			     xtest_conv.gsheet


In [9]:
from sklearn.model_selection import train_test_split

project_path = '/content/drive/My Drive/Colab Notebooks/CPSTN_NLP_GRP6A/'
file_name = 'data_modeled.csv'
conv_df = pd.read_csv (project_path + file_name)
print(' The shape of the Data Frame ' + str(conv_df.shape))
conv_df.head()
#df_trn, df_val = train_test_split(conv_df, stratify = conv_df['Assignment group'#], test_size = 0.2, random_state = 12)

 The shape of the Data Frame (8500, 13)


,Unnamed: 0,Unnamed: 0.1,Short description,Description,Caller,Assignment group,slen,dlen,Full Desc,language,Updated Lang,Full Len,Assignment group modified
0,0,0,login issue,-verified user details.(employee# & manager na...,spxjnwir pjlcoqds,GRP_0,11,206,login issue verified user detail employee mana...,en,en,198,GRP_0
1,1,1,outlook,\r\n\r\nreceived from: hmjdrvpb.komuaywn@gmail...,hmjdrvpb komuaywn,GRP_0,7,194,outlook received from hello team my meeting sk...,en,en,154,GRP_0
2,2,2,cant log in to vpn,\r\n\r\nreceived from: eylqgodm.ybqkwiam@gmail...,eylqgodm ybqkwiam,GRP_0,18,87,cant log in to vpn received from hi cannot log...,en,en,61,GRP_0
3,3,3,unable to access hr_tool page,unable to access hr_tool page,xbkucsvz gcpydteq,GRP_0,29,29,unable to access hr_tool page,en,en,29,GRP_0
4,4,4,skype error,skype error,owlgqjme qhcozdfx,GRP_0,12,12,skype error,no,no,11,GRP_0


In [10]:
# conv_df['labels'] = '__label__' + conv_df['Assignment group modified']
# list_of_tuples = list(zip(conv_df['labels'], conv_df['Full Desc']))  
# df = pd.DataFrame(list_of_tuples, columns = ['Assignment group', 'Description'])  
conv_df['labels'] = '__label__' + conv_df['Assignment group']
list_of_tuples = list(zip(conv_df['labels'], conv_df['Description']))  
df_ft = pd.DataFrame(list_of_tuples, columns = ['Assignment group', 'Description'])  
df_ft.head()

,Assignment group,Description
0,__label__GRP_0,-verified user details.(employee# & manager na...
1,__label__GRP_0,\r\n\r\nreceived from: hmjdrvpb.komuaywn@gmail...
2,__label__GRP_0,\r\n\r\nreceived from: eylqgodm.ybqkwiam@gmail...
3,__label__GRP_0,unable to access hr_tool page
4,__label__GRP_0,skype error


In [11]:
#df_ft.dropna(inplace=True)
df_ft.isna().sum()

Assignment group    0
Description         0
dtype: int64

In [13]:
top_5 = list(df_ft.groupby("Assignment group").size().reset_index().sort_values(by=0,ascending=False)[:5]["Assignment group"].values)
df_ft_5 = df_ft[df_ft["Assignment group"].isin(top_5)].copy(deep=True)

In [14]:
#X_train, X_test, Y_train, Y_test = train_test_split(df, df['Assignment group'], test_size = 0.3)
X_train, X_test, Y_train, Y_test = train_test_split(df_ft_5, df_ft_5['Assignment group'], test_size = 0.2)
X_train_v, X_val, Y_train_v, Y_val = train_test_split(X_train, X_train['Assignment group'], test_size = 0.20)

In [15]:
print(X_train.shape,X_train_v.shape, X_val.shape,X_test.shape)

(4348, 2) (3478, 2) (870, 2) (1087, 2)


In [16]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4348 entries, 1223 to 652
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Assignment group  4348 non-null   object
 1   Description       4348 non-null   object
dtypes: object(2)
memory usage: 101.9+ KB


In [17]:
# ENTIRE TRAIN DATA
np.savetxt('ft_train_2.txt', X_train.values, fmt='%s')

#TRAIN DATA FOR VALIDATION
np.savetxt('ft_train_v2.txt', X_train_v.values, fmt='%s')
#VALIDATION SET
np.savetxt('ft_val2.txt', X_val.values, fmt='%s')

# Test set
np.savetxt('ft_test2.txt', X_test.values, fmt='%s')

In [ ]:
#Remove
# import fasttext
# import os
# os.chdir("/content/drive/My Drive/CPSTN_NLP_GRP6A/")

In [18]:
model = fasttext.train_supervised(input='ft_train_v2.txt', autotuneValidationFile='ft_val2.txt')

In [20]:
project_path = '/content/drive/My Drive/Colab Notebooks/CPSTN_NLP_GRP6A/'
model.save_model(project_path+ 'fasttext_auto_tune_accuracy.bin')

In [ ]:
#Tunes model only for group 0
model_at = fasttext.train_supervised(input='ft_train_1.txt', autotuneValidationFile='ft_train_v1.txt', autotuneMetric="f1:__label__GRP_0") 

In [21]:
model_lrpt1_e50 = fasttext.train_supervised(input="ft_train_2.txt", lr=0.05, epoch=500) #91.2%

In [24]:
model_lrpt1_e50.save_model(project_path+ 'fasttext_lrpt05_epoch500.bin')

In [22]:
model_lrpt1_e50.test("ft_test2.txt")

(1087, 0.9126034958601656, 0.9126034958601656)

In [32]:
model.predict('Schedule maintanence of remote dial',k=2)

(('__label__GRP_8', '__label__GRP_9'), array([0.26118717, 0.22911464]))

In [26]:
model_lrpt1_e50.predict('job job_504 failed in job_scheduler')

(('__label__GRP_8',), array([0.94708002]))

In [27]:
model_lrpt1_e50.predict('circuit outage')

(('__label__GRP_0',), array([0.89840502]))

In [ ]:
model_lrpt1_e50.save_model('fasttext_main.bin')

In [ ]:
model_lr1_e10 = fasttext.train_supervised(input="ft_train_2.txt", lr=1,epoch=10)

In [ ]:
model_lr1_e10.test("ft_test2.txt")

(1087, 0.8886844526218951, 0.8886844526218951)

In [ ]:
model_lr1_e100 = fasttext.train_supervised(input="ft_train_2.txt", lr=0.1,epoch=20, bucket=200000, dim=50, loss='hs')

In [ ]:
model_lr1_e100.test("ft_test2.txt")

(1087, 0.8942042318307267, 0.8942042318307267)

In [ ]:
model_lr1_e100.predict('unable access machine utility finish drawer adjustment setting network')

(('__label__GRP_0',), array([0.9812935]))

In [ ]:
model_lr1_e100.predict('job job_504 failed in job_scheduler')

(('__label__GRP_0',), array([0.54923636]))

In [ ]:
model_lr1_e100_ngrams = fasttext.train_supervised(input="ft_train_2.txt", lr=0.1,epoch=20, wordNgrams=2, bucket=200000, dim=50, loss='hs')

In [ ]:
model_lr1_e100_ngrams.test("ft_test2.txt")

(1087, 0.890524379024839, 0.890524379024839)

In [ ]:
model_plain = fasttext.train_supervised(input="ft_train_2.txt")
model_plain.test("ft_test2.txt")

(1087, 0.874885004599816, 0.874885004599816)

In [19]:
model.test("ft_test2.txt")

(1087, 0.9116835326586936, 0.9116835326586936)